In [ ]:
!pip install simpletransformers


In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import classification_report

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

import re


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Import Data

In [ ]:
#Creating the dataframes
kiva_train = pd.read_csv('https://raw.githubusercontent.com/DDave94/Loan-Default-Prediction/main/kiva_train.csv')
kiva_test = pd.read_csv('https://raw.githubusercontent.com/DDave94/Loan-Default-Prediction/main/kiva_test.csv')
kiva_train.head(10)

,loan_id,en_clean,defaulted
0,7779,She opened a colmado out of the side of her ho...,0
1,2777,(First Loan): Joffre continues to run his loc...,1
2,6007,"Dina Santana is the mother of two children, Ju...",0
3,76,"Rosemary is 50 years old, single, and has 6 ch...",1
4,4217,"Segundo has a shop where he sells animal feed,...",0
5,5077,"I am a single parent, mother of 4. I sell groc...",0
6,6033,Mariana Jose Serda owns a general store in Mag...,0
7,843,Mary is a mother of four children. One of the ...,1
8,4357,Wilson is seeking his second loan with Kiva an...,0
9,7007,Clara lives with her two children (ages: 20 an...,0


### Preprocessing

In [ ]:
pip install unidecode textstat

### Looking at Training Data after preprocessing

In [ ]:
kiva_train.head(10)

,loan_id,en_clean,defaulted
0,7779,She opened a colmado out of the side of her ho...,0
1,2777,(First Loan): Joffre continues to run his loc...,1
2,6007,"Dina Santana is the mother of two children, Ju...",0
3,76,"Rosemary is 50 years old, single, and has 6 ch...",1
4,4217,"Segundo has a shop where he sells animal feed,...",0
5,5077,"I am a single parent, mother of 4. I sell groc...",0
6,6033,Mariana Jose Serda owns a general store in Mag...,0
7,843,Mary is a mother of four children. One of the ...,1
8,4357,Wilson is seeking his second loan with Kiva an...,0
9,7007,Clara lives with her two children (ages: 20 an...,0


### Train / Val Data

In [ ]:
# Creating Training Data
X = kiva_train['en_clean']
y = kiva_train['defaulted']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_df = pd.concat([X_train,y_train], axis = 1)
eval_df = pd.concat([X_val,y_val],axis = 1)

### DL Model

In [ ]:
# Create arguements
model_args = ClassificationArgs()
model_args.overwrite_output_dir = True
model_args.no_cache = True
model_args.save_model_every_epoch = False
model_args.num_train_epochs = 8
model_args.learning_rate = 4e-5
model_args.train_batch_size = 32
model_args.eval_batch_size = 32
model_args.max_seq_length = 300
model_args.manual_seed = 20

#20 (0.9),140(0.89), without lower case (0.86), with lower case(0.87)
#seed 20 with lower case (0.87) (Lower case sucks)

#Seed 20 with pure lemmitization (0.88)
# Seed 20 with lemmitization and removing stop words (0.87)
# Seed 20 with just lower case (0.73)

#seed 20 with pure lemmitization and lower case, trying test size = 0.1 (0.71)

#Trying 70-30 split 0.87

# Create a Classification Model
model = ClassificationModel("bert", "bert-base-cased", args=model_args)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/4910 [00:00<?, ?it/s]

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Running Epoch 0 of 8:   0%|          | 0/154 [00:00<?, ?it/s]

Running Epoch 1 of 8:   0%|          | 0/154 [00:00<?, ?it/s]

...

Running Epoch 6 of 8:   0%|          | 0/154 [00:00<?, ?it/s]

Running Epoch 7 of 8:   0%|          | 0/154 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/1228 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/39 [00:00<?, ?it/s]

In [ ]:
preds, outputs = model.predict(X_val.to_list())

  0%|          | 0/1228 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

In [ ]:
print(classification_report(y_val, preds))

              precision    recall  f1-score   support

           0       0.88      0.89      0.88       629
           1       0.88      0.87      0.87       599

    accuracy                           0.88      1228
   macro avg       0.88      0.88      0.88      1228
weighted avg       0.88      0.88      0.88      1228



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_val, preds))

[[558  71]
 [ 79 520]]
